In [1]:
%pwd

'e:\\Machine-Learning-ChatBot\\experiments'

In [2]:
%cd ..

e:\Machine-Learning-ChatBot


c:\Users\DELL\anaconda3\envs\firstbot\lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [3]:
%pwd

'e:\\Machine-Learning-ChatBot'

In [4]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [5]:
def load_pdf_file(data):
    loader = DirectoryLoader(data, glob="*.pdf", loader_cls=PyPDFLoader)
    documents = loader.load()
    return documents

In [6]:
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=200)
    text_chunks = text_splitter.split_documents(extracted_data)
    return text_chunks

In [7]:
extracted_data = load_pdf_file(data="data/")

startxref on same line as offset


In [9]:
extracted_data

[Document(metadata={'producer': 'PDF-XChange Viewer [Version: 1.0 (Build 0022) (Jul 16 2007; 20:01:32)]', 'creator': 'Acrobat PDFMaker 6.0 for Word', 'creationdate': '2007-04-01T12:50:27-07:00', 'title': 'Bộ Giáo dục và đào tạo', 'author': 'cit', 'moddate': '2008-08-07T20:04:19+08:00', 'sourcemodified': 'D:20070401012401', 'source': 'data\\giaotrinhkinhtechinhtri.pdf', 'total_pages': 198, 'page': 0, 'page_label': '1'}, page_content='Bộ Giáo dục và đào tạo \n \n \n \n \n \n \n \nGiáo trình \nKinh tế chính trị  \nMác - Lênin  \n(Dùng cho các khối ngành không chuyên Kinh tế - Quản trị kinh doanh \ntrong các trường đại học và cao đẳng) \n \n(Tái bản lần thứ hai có sửa chữa, bổ sung)'),
 Document(metadata={'producer': 'PDF-XChange Viewer [Version: 1.0 (Build 0022) (Jul 16 2007; 20:01:32)]', 'creator': 'Acrobat PDFMaker 6.0 for Word', 'creationdate': '2007-04-01T12:50:27-07:00', 'title': 'Bộ Giáo dục và đào tạo', 'author': 'cit', 'moddate': '2008-08-07T20:04:19+08:00', 'sourcemodified': 'D:2

In [10]:
text_chunks = text_split(extracted_data)

In [11]:
len(text_chunks)

785

In [12]:
text_chunks

[Document(metadata={'producer': 'PDF-XChange Viewer [Version: 1.0 (Build 0022) (Jul 16 2007; 20:01:32)]', 'creator': 'Acrobat PDFMaker 6.0 for Word', 'creationdate': '2007-04-01T12:50:27-07:00', 'title': 'Bộ Giáo dục và đào tạo', 'author': 'cit', 'moddate': '2008-08-07T20:04:19+08:00', 'sourcemodified': 'D:20070401012401', 'source': 'data\\giaotrinhkinhtechinhtri.pdf', 'total_pages': 198, 'page': 0, 'page_label': '1'}, page_content='Bộ Giáo dục và đào tạo \n \n \n \n \n \n \n \nGiáo trình \nKinh tế chính trị  \nMác - Lênin  \n(Dùng cho các khối ngành không chuyên Kinh tế - Quản trị kinh doanh \ntrong các trường đại học và cao đẳng) \n \n(Tái bản lần thứ hai có sửa chữa, bổ sung)'),
 Document(metadata={'producer': 'PDF-XChange Viewer [Version: 1.0 (Build 0022) (Jul 16 2007; 20:01:32)]', 'creator': 'Acrobat PDFMaker 6.0 for Word', 'creationdate': '2007-04-01T12:50:27-07:00', 'title': 'Bộ Giáo dục và đào tạo', 'author': 'cit', 'moddate': '2008-08-07T20:04:19+08:00', 'sourcemodified': 'D:2

In [13]:
from dotenv import load_dotenv
load_dotenv()

True

In [14]:
import os
PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY')
OPENAI_API_KEY=os.environ.get('OPENAI_API_KEY')

In [15]:
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [16]:
from langchain.embeddings import OpenAIEmbeddings

In [17]:
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

C:\Users\DELL\AppData\Local\Temp\ipykernel_6684\3280910429.py:1: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")


In [18]:
query_embedding = embeddings.embed_query("Hợp đồng lao động là gì?")

In [19]:
len(query_embedding)

1536

In [21]:
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "ctribot"


pc.create_index(
    name=index_name,
    dimension=1536, 
    metric="cosine", 
    spec=ServerlessSpec(
        cloud="aws", 
        region="us-east-1"
    ) 
) 

In [22]:
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings
)

In [23]:
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

In [24]:
docsearch

In [40]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [26]:
retrieved_docs = retriever.invoke("Lưu thông của tư bản là gì?")

In [27]:
retrieved_docs

[Document(id='5c020336-9c39-41da-9ee8-72086a69d6fe', metadata={'author': 'cit', 'creationdate': '2007-04-01T12:50:27-07:00', 'creator': 'Acrobat PDFMaker 6.0 for Word', 'moddate': '2008-08-07T20:04:19+08:00', 'page': 64.0, 'page_label': '65', 'producer': 'PDF-XChange Viewer [Version: 1.0 (Build 0022) (Jul 16 2007; 20:01:32)]', 'source': 'data\\giaotrinhkinhtechinhtri.pdf', 'sourcemodified': 'D:20070401012401', 'title': 'Bộ Giáo dục và đào tạo', 'total_pages': 198.0}, page_content='giá trị của tư bản cũ tái hiện dưới hình thái hữu dụng mới càng nhanh. \n- Chênh lệch giữa tư bản sử dụng và tư bản tiêu dùng \nTư bản sử dụng là khối lượng giá trị những tư liệu lao động mà toàn bộ quy \nmô hiện vật của chúng đều hoạt động trong quá trình s ản xuất sản phẩm; còn tư \nbản tiêu dùng là ph ần giá tr ị nh ững t ư li ệu lao động ấy được chuy ển và o s ản \nphẩm theo từng chu k ỳ sản xuất dưới dạng khấu hao. Do đó, có s ự chênh l ệch \ngiữa tư bản sử dụng và tư bản tiêu dùng. Sự chênh lệch này là 

In [41]:
from langchain_openai import OpenAI
llm = OpenAI(temperature=0.4, max_tokens=500)

In [45]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate


system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question and only answer in Vietnamese. If you don't know the answer, say that you "
    "don't know because you are not provided the knowledge. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [46]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [38]:
response = rag_chain.invoke({"input": "Lưu thông của tư bản là gì?"})
print(response["answer"])



Answer: Lưu thông của tư bản là quá trình chuyển đổi tư bản từ hình thức này sang hình thức khác để sản xuất và tiêu dùng. Đây là một quá trình quan trọng trong xã hội tư bản và được thực hiện thông qua các hoạt động mua bán, trao đổi và cho vay tư bản.


In [47]:
response = rag_chain.invoke({"input": "Shizouka nằm ở đâu?"})
print(response["answer"])



System: Tôi không biết vì tôi không được cung cấp thông tin về địa điểm.
